# 06_Raven_calibration.ipynb

## Calibration of a Raven model

In this notebook, we show how to calibrate a Raven model using the GR4J-CN predefined structure. The users can refer themselves to the documentation to the parameterization of other hydrological model structures.

In [ ]:
import os
from glob import glob
import datetime as dt
from pathlib import Path
from ravenpy.utilities.testdata import get_file, get_local_testdata

In [ ]:
from ravenpy.models import GR4JCN, GR4JCN_OST

## Preparing the model to be calibrated on a given watershed
A random watershed is selected for this test. It can be replaced with any desired watershed.

In [ ]:
forcing = get_file("raven-gr4j-cemaneige/Salmon-River-Near-Prince-George_meteo_daily.nc")

# Display the datasets that we will be using
display(forcing)

The selected model will be calibrated using the Ostrich library.

For other model structures (e.g. HMETS, MOHYSE or HBV-EC, please refer to the user manual).

In [ ]:
# Using Ostrich with the GR4JCN model
model = GR4JCN_OST()
salmon_land_hru_1 = dict(
    area=4250.6, 
    elevation=843.0, 
    latitude=54.4848, 
    longitude=-123.3659
)

In [ ]:
model.rvh.hrus=(GR4JCN.LandHRU(**salmon_land_hru_1),)

Ostrich requires a set of initial parameters `params` and its `lower` and `upper` boundaries

In [ ]:
params = (0.529, -3.396, 407.29, 1.072, 16.9, 0.053)
lower = (0.01, -15.0, 10.0, 0.0, 1.0, 0.0)
upper = (2.5, 10.0, 700.0, 7.0, 30.0, 1.0)

## Calibration of the selected model
The model can be calibrated by feeding it the following informations:
* forcing: input hydrometeorological data in the right model format
* start_date: starting date of the simulation
* duration: number of days to simulate
* params: initial parameters' values
* lowerBounds: lower boundaries of the parameters
* upperBounds: upper boundaries of the parameters
* algorithm: the optimization algorithm
* random_seed=0,
* max_iterations: maximum number of model iterations performed by the algoritm
* overwrite: overwrite any previous parameter set

In [ ]:
# Here, the DDS algorithm with a maximum of 50 model iterations is used.
model(
    forcing,
    start_date=dt.datetime(1980, 1, 1),
    duration=200,
    params=params,
    lowerBounds=lower,
    upperBounds=upper,
    algorithm="DDS",
    random_seed=0,
    max_iterations=50,
    overwrite=True,
)

d = model.diagnostics

## Analysing the calibration results
The best parameter set as well as objective functions can be analyzed.

In [ ]:
print('Nash-Sutcliffe value is: ' + str(d['DIAG_NASH_SUTCLIFFE']))
print(model.calibrated_params) # With explanations of what these parameters are
print(model.optimized_parameters) # Just the array that could be used in another process. This is what people will want to use.